In [4]:
import openai 
import pandas as pd
import re
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=openai_api_key)

In [5]:
df1=pd.read_csv("processed_data.csv")
df2=pd.read_csv("processed_data2.csv")

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            83 non-null     object
 1   webVideoUrl     100 non-null    object
 2   diggCount       100 non-null    int64 
 3   shareCount      100 non-null    int64 
 4   playCount       100 non-null    int64 
 5   collectCount    100 non-null    int64 
 6   commentCount    100 non-null    int64 
 7   duration        100 non-null    int64 
 8   musicName       100 non-null    object
 9   musicAuthor     100 non-null    object
 10  nickName        100 non-null    object
 11  fans            100 non-null    int64 
 12  hashtags        97 non-null     object
 13  hashtag_count   100 non-null    int64 
 14  bigrams         100 non-null    object
 15  trigrams        100 non-null    object
 16  Sentiment_text  100 non-null    object
dtypes: int64(8), object(9)
memory usage: 13.4+ KB


In [7]:
df2.head()

,contenido,contenido_limpio,bigrams
0,Ya vamos a hacer rating de las galletas que ve...,vamos hacer rating galletas venden peru aburri...,"['vamos hacer', 'hacer rating', 'rating gallet..."
1,"Hola chicas, buenas noches. Y de veras que son...",hola chicas buenas noches veras noches 12 noch...,"['hola chicas', 'chicas buenas', 'buenas noche..."
2,calificando las galletas de Joy Bakery con mi ...,calificando galletas joy bakery mejor amiga ca...,"['calificando galletas', 'galletas joy', 'joy ..."
3,"Buenas, cómo andan? Vamos a hacer unas cracke...",buenas com andan vamos hacer unas cracker crac...,"['buenas com', 'com andan', 'andan vamos', 'va..."
4,Hoy les enseño a hacer las mejores chocolate c...,hoy enseñ hacer mejores chocolate chip cooky s...,"['hoy enseñ', 'enseñ hacer', 'hacer mejores', ..."


In [8]:
df1.head()

,text,webVideoUrl,diggCount,shareCount,playCount,collectCount,commentCount,duration,musicName,musicAuthor,nickName,fans,hashtags,hashtag_count,bigrams,trigrams,Sentiment_text
0,beautiful blue thought,https://www.tiktok.com/@kerrysbouqcakes/video/...,173800,7256,3000000,15700,313,10,Kulture Klub (Sped up Version),Lucy Tun,Kerry's Bouqcakes,411000,#beautifulblue #blue #buttercream #buttercream...,17,"['beautiful blue', 'blue thought']",['beautiful blue thought'],Positive
1,whats favourite thing christmas long go certai...,https://www.tiktok.com/@kerrysbouqcakes/video/...,17600,2263,486900,2296,863,49,Christmas,neozilla,Kerry's Bouqcakes,411000,#buttercream #floralcupcakes #floralpiping #cu...,15,"['whats favourite', 'favourite thing', 'thing ...","['whats favourite thing', 'favourite thing chr...",Positive
2,NaN,https://www.tiktok.com/@cupcakesloversv/video/...,10,0,173,0,0,8,Run Free (Original Mix),Deep Chills,Cupcakes Lover,190,#cupxakes #proceso #cremaverde,3,[],[],Neutral
3,bday ceo konfidinz thanks cupcakesthey taste good,https://www.tiktok.com/@brebre0029/video/74258...,7,1,92,0,3,29,SJAYY Not Like Us FT. Kendrick Lamar,SJAYY,Bre Bre,580,#cupcakelovers #sweets #bdaygirl #bdayqueen #v...,5,"['bday ceo', 'ceo konfidinz', 'konfidinz thank...","['bday ceo konfidinz', 'ceo konfidinz thanks',...",Positive
4,NaN,https://www.tiktok.com/@cupcakesloversv/video/...,257,2,2137,1,0,15,Mega Francesita (Remix),DJ Dmonio,Cupcakes Lover,190,#pastelespersonalizados #parati #cupcakeslover,3,[],[],Neutral


In [9]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')


## Recomendaciones de descripciones
En este apartado mandaremos el dataframe1 y filtrará por vistas los videos y hará una recomendación sobre cual seria la descripción ideal

In [10]:
def recomendacion_descripcion(df: pd.DataFrame, cantidad: int, api_key: str) -> str:
    df_filter = df[df["playCount"] > cantidad]
    text = " ".join(df_filter["text"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Estás a cargo del sector de marketing."},
                {"role": "user", "content": f"Below you have all the descriptions of videos that have a number of views greater than {cantidad}, which is the following {text}, I want you to analyze everything and return me the ideal description to create a high-impact video that produces great number of views. (Give me the description in Spanish please) (It is a description of TikTok, it should not be large in size) The characters should not exceed 50-100 [just give me the description, don't give me any more extra]"}
            ],
            # max_tokens=150
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en recomendacion_descripcion: {e}")
        return f"Error: {str(e)}"
    

recomendacion=recomendacion_descripcion(df1,10000,openai_api_key)
print(recomendacion)

2024-11-07 10:26:13,561 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


¡Descubre cómo hacer los cupcakes más deliciosos! 🍰 Aprende a decorar con recetas fáciles y sorprende a todos. ¡No olvides compartir tu creación! #CupcakeLover #RecetasDeliciosas


## Ideas principales de los videos

In [11]:
def ideas_principales(df: pd.DataFrame, cantidad: int, num_ideas: int, api_key: str) -> str:
    df_filter = df[df["playCount"] > cantidad]
    text = " ".join(df_filter["text"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Estás a cargo del sector de marketing."},
                {"role": "user", "content": f"Below you have all the descriptions of videos that have a number of views greater than {cantidad}, which is the following {text}, I want you to analyze these descriptions and tell me what are the {num_ideas} main ideas that the videos have (Give me the description in Spanish please) "}
            ],
            # max_tokens=150
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en ideas_principales: {e}")
        return f"Error: {str(e)}"

ideas_prin=ideas_principales(df1,10000,3,openai_api_key)
print(ideas_prin)

2024-11-07 10:26:38,897 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Claro, aquí tienes las tres principales ideas de las descripciones de los videos, en español:

1. **Recetas de Cupcakes**: Las descripciones mencionan diversas recetas de cupcakes, incluyendo ingredientes y pasos para preparar cupcakes de vainilla, chocolate y otros sabores, enfatizando la importancia de las técnicas de decoración con buttercream.

2. **Tutoriales y Clases de Decoración**: Se destaca la disponibilidad de tutoriales en línea para aprender a decorar cupcakes y crear ramos de cupcakes, invitando a los espectadores a visitar sitios web específicos para mejorar sus habilidades de repostería.

3. **Celebraciones y Ocasiones Especiales**: Las descripciones hacen referencia a ocasiones especiales como la Navidad y el Día Nacional del Amante de los Cupcakes, sugiriendo que los cupcakes son una parte divertida y deliciosa de las celebraciones y momentos compartidos con seres queridos.


## Hashtags ideales

In [12]:
def hashtags_ideales(df: pd.DataFrame, cantidad: int, num_hash: int, api_key: str) -> list:
    df_filter = df[df["playCount"] > cantidad]
    hashtags = "|".join(df_filter["hashtags"].astype(str).values)
    views = "|".join(df_filter["playCount"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Eres un asistente de análisis de hashtags."},
                {"role": "user", "content": f"Provide only a list of the top {num_hash} most impactful hashtags based on this data: views {views} and hashtags {hashtags} [hashtags per video are separated by '|']. Ouput algo asi (Tus hashtags ideales son los siguientes #- , #-,etc)"}
            ],
            max_tokens=100
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en hashtags_ideales: {e}")
        return f"Error: {str(e)}"
 

hashtags = hashtags_ideales(df1, 10000, 3, openai_api_key)

print(hashtags)



2024-11-07 10:26:53,146 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tus hashtags ideales son los siguientes: #buttercream, #cupcakeinspo, #edibleart.


## Ideas principales basadas en transcripción

In [13]:
def ideas_principales2(df: pd.DataFrame, num_ideas: int, api_key: str) -> str:
    text = "|".join(df["contenido_limpio"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Estás a cargo del sector de marketing."},
                {"role": "user", "content": f"Below you have all the transcripts of the videos that have a good number of views, estas son las transcripciones del video {text} [separadas por '|'], quiero que analices estas transcripciones y me digas cuáles son las {num_ideas} ideas principales que tienen los videos (Give me the description in Spanish please) "}
            ],
            # max_tokens=150
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en ideas_principales2: {e}")
        return f"Error: {str(e)}"

ideas_prin2=ideas_principales2(df2,3,openai_api_key)
print(ideas_prin2)

2024-11-07 10:27:11,225 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1. **Calificaciones y Comparaciones de Galletas**: Muchos de los videos se centran en calificar y comparar diferentes tipos de galletas, tanto comerciales como caseras. Esto incluye mencionar marcas populares, sabores y características específicas que hacen que cada tipo de galleta sea especial o destacada. Este enfoque busca guiar a los espectadores sobre cuáles galletas probar según sus preferencias personales.

2. **Recetas de Galletas Caseras**: Una gran parte de los videos ofrece recetas para hacer galletas en casa, destacando los ingredientes y el proceso de preparación. Los creadores de contenido comparten recetas diversas, desde galletas de avena hasta galletas con chips de chocolate, lo que permite a los espectadores experimentar en su cocina y disfrutar de resultados caseros.

3. **Experiencias y Recomendaciones**: Los videos también incluyen recomendaciones y anécdotas personales sobre las galletas, ya sea en términos de sabor, textura o experiencias al hornear. Los creadore

## Ideas para videos

In [14]:
def ideas_video(df: pd.DataFrame, num_ideas: int, api_key: str) -> str:
    text = "|".join(df["contenido_limpio"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Estás a cargo del sector de marketing."},
                {"role": "user", "content": f"Below you have all the transcripts of the videos that have a good number of views, estas son las transcripciones del video {text} [separadas por '|'], quiero que analices las descripciones y me des {num_ideas} ideas para futuros videos de TikTok (Give me the description in Spanish please) "}
            ],
            # max_tokens=150
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en ideas_video: {e}")
        return f"Error: {str(e)}"

ideas=ideas_video(df2,3,openai_api_key)
print(ideas)

2024-11-07 10:27:27,096 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Claro, aquí tienes tres ideas para futuros videos de TikTok basados en las descripciones de los videos anteriores:

1. **Desafío de Galletas Rellenas**: "¡Únete al desafío de galletas rellenas! Vamos a probar diferentes combinaciones de galletas con rellenos sorprendentes. ¿Qué pasará cuando mezclemos oreo con manjar o chocolate blanco con galleta de vainilla? ¡Te sorprenderás! Comparte tus combinaciones favoritas en los comentarios y déjanos saber cuál te gustaría que probáramos en el próximo video. ¡Sigue para ver el resultado!"

2. **Recetas Saludables de Galletas**: "Hoy te traemos una receta fácil y saludable de galletas de avena y chocolate. Solo necesitas unos pocos ingredientes para hacer unas galletas deliciosas y nutritivas. Ideal para un snack entre comidas o para disfrutar sin culpa. Quédate hasta el final para ver el resultado y no olvides seguirnos para más recetas saludables."

3. **Galletas Épicas de Halloween**: "¡Halloween está a la vuelta de la esquina! Vamos a prepa

## Idea de Guion

In [15]:
def idea_guion(df: pd.DataFrame, topic: str, api_key: str) -> str:
    text = "|".join(df["contenido_limpio"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Estás a cargo del sector de marketing."},
                {"role": "user", "content": f"Below you have all the transcripts of the videos that have a good number of views, estas son las transcripciones del video {text} [separadas por '|'], quiero que analices las descripciones y me des una idea de guion para un futuro video de TikTok relacionado con el tema {topic} (Give me the description in Spanish please)"}
            ],
            # max_tokens=150
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en idea_guion: {e}")
        return f"Error: {str(e)}"
tema="cupcakes de navidad"
guion=idea_guion(df2,tema,openai_api_key)
print(guion)

2024-11-07 10:27:53,791 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**Guion para Video de TikTok: "Cupcakes Navideños Deliciosos y Creativos"**

**[Inicio del Video]**  
- **(Escena 1: Presentación)**  
  - Mostrar una mesa decorada con motivos navideños.
  - Hablar a cámara: “¡Hola, amantes de la cocina! Hoy les traigo una receta espectacular para hacer cupcakes navideños que van a deslumbrar en cualquier celebración. ¡Vamos a poner manos a la obra!”

**[Paso 1: Ingredientes]**  
- **(Escena 2: Mostrar Ingredientes)**  
  - Enfocar los ingredientes sobre la mesa.  
  - “Necesitamos: una taza de harina, media taza de azúcar, un cuarto de taza de mantequilla derretida, un huevo, un poco de polvo de hornear, leche, y lo más importante, un toque de canela y nuez moscada para ese sabor navideño.”

**[Paso 2: Mezcla de Ingredientes]**  
- **(Escena 3: Mezclando)**  
  - Mostrar cómo mezclar los ingredientes en un tazón.  
  - “Primero, mezclamos los ingredientes secos: la harina, el azúcar, el polvo de hornear, y las especias. Ahora, en otro recipiente, bat

## Dame un Resumen general de los videos

In [16]:
def resumen(df: pd.DataFrame, api_key: str) -> str:
    text = "|".join(df["contenido_limpio"].astype(str).values)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Estás a cargo del sector de marketing."},
                {"role": "user", "content": f"Below you have all the transcripts of the videos that have a good number of views, estas son las transcripciones del video {text} [separadas por '|'], quiero que analices los transcripts y me des un resumen general de los videos (Give me the description in Spanish please) [en formato de párrafo]"}
            ],
            # max_tokens=200
        )
        response = response.model_dump()
        return response['choices'][0]['message']['content']
    except Exception as e:
        logging.error(f"Error en resumen: {e}")
        return f"Error: {str(e)}"
resumen=resumen(df2,openai_api_key)
print(resumen)

2024-11-07 10:28:08,859 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


A lo largo de los videos transcritos, se presenta una variedad de recetas y calificaciones de galletas, muchas de las cuales son populares en Perú. Los videos incluyen desde la preparación de galletas de avena con pasas, galletas de chocolate saludables, hasta la calificación de diferentes tipos de galletas comerciales, como las Oreo y las galletas de la marca Ritz. Los creadores de contenido también comparten tips útiles para hornear, como la temperatura adecuada y el tiempo de cocción, así como ingredientes alternativos para versiones más saludables. Además, algunos videos destacan experiencias personales en la cocina y la importancia de disfrutar momentos en familia o con amigos al preparar y degustar estas delicias. También se mencionan productos específicos del mercado peruano, como galletas rellenas y opciones veganas, dejando ver un enfoque en la accesibilidad y la creatividad en la cocina. En resumen, estos videos ofrecen una mezcla atractiva de entretenimiento culinario y cons

## Insight personalizado

In [17]:
import pandas as pd
from openai import OpenAI
import logging

def columnas_ideales(df: pd.DataFrame, peticion: str, api_key: str) -> list:
    columnas = list(df.columns)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Eres un asistente de análisis de datos."},
                {"role": "user", "content": f"Here are the columns: {columnas}. Please suggest the best columns for this request: '{peticion}'. Only return them as a Python list in this format: ['column1', 'column2', ...]. No additional explanation."}
            ],
            max_tokens=100
        )
        response = response.model_dump()
        recommended_columns = eval(response['choices'][0]['message']['content'].strip())
        return recommended_columns
    except Exception as e:
        logging.error(f"Error en columnas_ideales: {e}")
        return []

def peticion_personalizada(df: pd.DataFrame, peticion: str, api_key: str) -> dict:
    
    columnas_recomendadas = columnas_ideales(df, peticion, api_key)

    existing_cols = [col for col in columnas_recomendadas if col in df.columns]

    if not existing_cols:
        return {"analysis": "Error: The provided DataFrame is empty. Please provide a non-empty DataFrame to analyze.", "num_columns": 0, "num_rows": 0}

    df_recomendado = df[existing_cols]
    info_columnas = df_recomendado.to_string(index=False)

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a data analysis assistant."},
                {"role": "user", "content": f"Here is data for the relevant columns for this request: '{peticion}'.\n\n{info_columnas}\n\nPlease give me the answer of the request .(Give me the description in Spanish please)"}
            ]
        )
        analysis = response.choices[0].message.content
        return {"analysis": analysis, "num_columns": len(existing_cols), "num_rows": len(df_recomendado)}
    except Exception as e:
        logging.error(f"Error in peticion_personalizada: {str(e)}")
        return {"analysis": "Error: Unable to perform analysis.", "num_columns": 0, "num_rows": 0}    


peticion="quiero que me digas que topicos puedo hacer para mis futuros videos"
peticion_perso=peticion_personalizada(df1,peticion,openai_api_key)
print(peticion_perso)

2024-11-07 10:28:27,584 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-07 10:28:41,085 INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'analysis': 'Claro, aquí tienes algunas ideas de tópicos para tus futuros videos relacionados con cupcakes y pasteles, basadas en los datos que proporcionaste:\n\n1. **Técnicas de decoración de cupcakes**: Muestra diferentes técnicas de decoración utilizando crema de mantequilla y otros ingredientes. Puedes incluir tutoriales paso a paso.\n\n2. **Recetas de cupcakes**: Comparte recetas únicas de cupcakes, como cupcakes de chocolate, vainilla, y otros sabores interesantes, como peras o limones.\n\n3. **Cupcakes temáticos**: Realiza cupcakes inspirados en festividades o eventos, como Halloween, Navidad o cumpleaños. Puedes mostrar cómo decorarlos de manera creativa.\n\n4. **Cupcakes saludables**: Explora recetas de cupcakes más saludables o con ingredientes bajos en calorías, perfectos para quienes buscan o controlar su dieta.\n\n5. **Decoración floral en cupcakes**: Da un tutorial sobre cómo hacer decoración de flores en cupcakes utilizando técnicas de repostería, como el uso de boquil